In [1]:
#Machine Learning and Kaggle: Titanic 

#In the Titanic we were given a test data set and using decision trees and sklearn I was able to compute a percentage 
#of which people would die or survive. I did this by reading in the test data and train data.
#I then began to seperate the entities into a table by specific attributes such as the name, ticket, and cabin for the 
#passengers aboard on the titanic. Then I dropped any information that was not available.
#Once I dropped the information that I did not need and was unavailable, I then began to fill in the missing values
#for the ages to have some values in this position. I filled in the missing values with the median of all the 
#titanic data provided to me. Once I completed filling in the table I cross validated the data with the test
#and train data using sklearn. From that I was able to generate a percentage of the amount of correct data
#I had compared to the train data. Then I compared my results to the test data set.

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from scipy.stats import mode
import string
import os

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train_test = [train, test]
train.shape

train.head()

train.describe()

cols = ['Name','Ticket','Cabin']
train = train.drop(cols,axis=1)

train.head()

train = train.dropna()

titanic_ds = pd.concat(ds, axis=1)

train = pd.concat((train,titanic_ds),axis=1)

train = train.drop(['Pclass','Sex','Embarked'],axis=1)

train['Age'].fillna(train['Age'].median(), inplace=True)

X = train.values
y = train['Survived'].values

X = np.delete(X,1,axis=1)

from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.40,random_state=0)

from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=5)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

test['Survived'] = 0
test.loc[test['Sex'] == 'female','Survived'] = 1
data_to_submit = pd.DataFrame({
    'PassengerId':test['PassengerId'],
    'Survived':test['Survived']
})

In [2]:
# Maching Learning and Kaggle: Digit Recognition

# Using sklearn, decomposition and linear discriminant analysis I was able to generate a percentage of 
# correct data from the Digit Recognition competition from Kaggle. I first began reading in the train data.
# I then began to display the data given and began to plot a whole slice of the pictures. During this time 
# I also began to plot the numbers that were displayed in a row by seperating the label and the pixels. 
# I then began to drop the labels because they were not needed and I split up the data for the model selection.
# Once this was completeed I used the principal component analysis for the linear dimensionality reduction.
# From this point I was able to transform my data sets using the dataFrame and using plotting a whole slice of pictures
# At this point I was able to extract the best score and best parameter

In [ ]:
import numpy as np
import pandas as pd

import os
print(os.listdir("../input"))

import seaborn as sns
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from time import time

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import math

# Plots image which is displayed in a row
def plot_num(row, w=28, h=28, labels=True):
    if labels:
        # first column is the label
        label = row[0]
        # all other columns are pixels
        pixels = row[1:]
    else:
        label = ''
        # all other columns are pixels
        pixels = row[0:]
        

    # Creating array of 8-bits pixels from the columns
    # 1D array of length 784
    # The pixel intensity values are integers from 0 to 255
    pixels = 200-np.array(pixels, dtype='uint8')

    # Reshaping array into 2D, 28 x 28 array
    pixels = pixels.reshape((w, h))

    # Plot
    if labels:
        plt.title('Label is {label}'.format(label=label))
    plt.imshow(pixels, cmap='gray')

# Plotting a whole slice of pictures
def plot_slice(rows, size_w=28, size_h=28, labels=True):
    num = rows.shape[0]
    w = 4
    h = math.ceil(num / w)
    fig, plots = plt.subplots(h, w)
    fig.tight_layout()

    for n in range(0, num):
        s = plt.subplot(h, w, n+1)
        s.set_xticks(())
        s.set_yticks(())
        plot_num(rows.ix[n], size_w, size_h, labels)
    plt.show()

#reading in train data
train = pd.read_csv('../input/train.csv')

#displaying data
print(train.shape)
train.sample(5)

train.describe()

train.info()



# Plotting first few rows
plot_slice(train[0:12])

#dropping labels
X_train = train.drop(['label'], axis='columns', inplace=False)
y_train = train['label']

#using sklearn to split data for model selection
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(X_train, y_train, test_size=0.30, random_state=4)


#using decompisition from sklearn for randomization
n_components = 16
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

#transforming data from train set
X_train_pca = pca.transform(X_train)


plt.hist(pca.explained_variance_ratio_, bins=n_components, log=True)
pca.explained_variance_ratio_.sum()


plot_slice(pd.DataFrame(data=X_train_pca[0:12]), size_w=4, size_h=4, labels=False)


param_grid = { "C" : [0.1]
              , "gamma" : [0.1]}
rf = SVC()
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=2, n_jobs=-1, verbose=1)
gs = gs.fit(X_train_pca, y_train)

print(gs.best_score_)
print(gs.best_params_)

bp = gs.best_params_

t0 = time()
clf = SVC(C=bp['C'], kernel='rbf', gamma=bp['gamma'])
clf = clf.fit(X_train_pca, y_train)
print("done in %0.3fs" % (time() - t0))

clf.score(pca.transform(X_ts), y_ts)


val = pd.read_csv('../input/test.csv')
pred = clf.predict(pca.transform(val))
# ImageId,Label

val['Label'] = pd.Series(pred)
val['ImageId'] = val.index +1
sub = val[['ImageId','Label']]

sub.to_csv('submission.csv', index=False)